In [ ]:
import aiohttp
import asyncio
import openai
from tqdm.notebook import tqdm
import time
import json
from sklearn.metrics import f1_score

In [ ]:
with open('../../data/screen_eval.json', 'r') as file:
    data = json.load(file)

In [ ]:
api_key = # key here
api_org = # org key here

openai.api_key = api_key
openai.organization = api_org

async def request(prompt, model='gpt-3.5-turbo'):
    async with aiohttp.ClientSession() as aiohttp_client_session:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1,
        )
        result = response.choices[0].message['content']
        print(result)
        return result

In [ ]:
binary_answer =[]
correct_answers=[]
clean_time=[]
messy_time=[]

for idx in tqdm(range(len(data['convo']))):
    t0m = time.time()
    decrease=0
    while True:
        t0=time.time()
        convo = ' '.join(data['original_convo'][str(idx)].replace('\n', ' \n').split(' ')[:2600-decrease])
        prompt = f"""{convo}

        Question: does the previous conversation factually imply "{data['inferred_summary'][str(idx)]}"? Answer Yes or No.
        """
        response = await request(prompt, model='gpt-3.5-turbo')
        sr = response.lower()
        binary_answer.append('yes' in sr)
        correct_answers.append(data['agg_label'][str(idx)])
        t1=time.time()
        time.sleep(2)
        break

    t1m = time.time()
    clean_time.append(t1-t0)
    messy_time.append(t1m-t0m)

with open('chatgpt_eval_results.json', 'w') as file:
    json.dump([binary_answer, correct_answers, clean_time, messy_time], file)

In [ ]:
f1_score(correct_answers, binary_answer, average='macro')

In [ ]:
binary_answer =[]
correct_answers=[]
clean_time=[]
messy_time=[]

for idx in tqdm(range(len(data['convo']))):
    t0m = time.time()
    decrease=0
    while True:
        t0=time.time()
        convo = ' '.join(data['original_convo'][str(idx)].replace('\n', ' \n').split(' ')[:2600-decrease])
        prompt = f"""{convo}

        Question: does the previous conversation factually imply "{data['inferred_summary'][str(idx)]}"? Answer Yes or No.
        """
        response = await request(prompt, model='gpt-4')
        sr = response.lower()
        binary_answer.append('yes' in sr)
        correct_answers.append(data['agg_label'][str(idx)])
        t1=time.time()
        time.sleep(2)
        break

    t1m = time.time()
    clean_time.append(t1-t0)
    messy_time.append(t1m-t0m)

with open('gpt_4_eval_results.json', 'w') as file:
    json.dump([binary_answer, correct_answers, clean_time, messy_time], file)

In [ ]:
f1_score(correct_answers, binary_answer, average='macro')